In [ ]:
import datetime as dt
import pandas as pd
import os
import glob
import json

In [ ]:
def mkNestedDir(dirTree):
    from pathlib import Path
    Path(dirTree).mkdir(parents=True, exist_ok=True)

In [ ]:
def write_subcells(number_of_cells, cell_id, spatial_resolution, east, west, elevation, pathout):
    header = "{number_of_cells} {spatial_resolution} {spatial_resolution}"
    header = header.format(number_of_cells=number_of_cells, spatial_resolution=spatial_resolution)

    cell_metadata = "{id} 1"
    cell_metadata = cell_metadata.format(id=cell_id, spatial_resolution=spatial_resolution)

    coordinates = "{east} {west} {elevation} 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0"
    coordinates = coordinates.format(east=east, west=west, elevation=elevation)

    file = open(pathout+"subcells.in","w+")
    [file.writelines(el + "\n") for el in [header, cell_metadata, coordinates]]

In [ ]:
def write_geometry(number_of_cells, spatial_resolution, pathout):
    header = "{number_of_cells} {spatial_resolution}"
    header = header.format(number_of_cells=number_of_cells, spatial_resolution=spatial_resolution)

    file = open(pathout+"geometry.in","w+")
    [file.writelines(el + "\n") for el in [header]]

In [ ]:
def write_params(variable, number_of_stations, kriging_type, kriging_model, kriging_correction, nugget, var, Ia, secs_timestep, starting_date, basin, pathout):

    line_1 = "{Ia}  0   0   {var}   {nug}               ! Parameters for kriging (I_a, I_e, m, var, var_nug) --> variogr. precip."
    line_2 = "{Ia}  0   0   {var}   {nug}               ! Parameters for kriging (I_a, I_e, m, var, var_nug) --> variogr. temper."
    line_3 = "-999  -999    -999    -999    -999        ! Parameters for kriging (I_a, I_e, m, var, var_nug) --> TMIN"
    line_4 = "-999  -999    -999    -999    -999        ! Parameters for kriging (I_a, I_e, m, var, var_nug) --> TMAX"
    line_5 = "{number_of_stations}                      ! nk: number of neighbour meteorological stations used in the kriging"
    line_6 = "{kriging_type}                            ! Type of Kriging OK, OKED"
    line_7 = "{kriging_model}                           ! Model equation EXP, DBLE_EXP --> modello di semivariogramma"
    line_8 = "{kriging_correction}				        ! Correction of negative weights NO, RESET, ADD, DEUTSCH --> pesi negativi ogni tanto, per avere sum(pesi)=1"
    line_9 = "{secs_timestep}                           ! Output time step [s]"
    line_10 = "{var}                                    ! P = Rainfall,  T = Temperature"
    line_11 = "{datetime}	                            ! Starting date (measurements) yyyy/mm/dd hh:mm"
    line_12 = "{basin}                                  ! Bacino"
    line_13 = "0					                    ! flag_varcond: 1--> evaluate conditional variance (0 --> otherwise)"

    if variable == "temperature":
        line_1 = line_1.format(Ia="-999", var="-999", nug="-999")
        line_2 = line_2.format(Ia=Ia, var=var, nug=nugget)
        line_10 = line_10.format(var="T")

    if variable == "precipitation":
        line_2 = line_2.format(Ia="-999", var="-999", nug="-999")
        line_1 = line_1.format(Ia=Ia, var=var, nug=nugget)
        line_10 = line_10.format(var="P")

    line_5 = line_5.format(number_of_stations=number_of_stations)
    line_6 = line_6.format(kriging_type=kriging_type)
    line_7 = line_7.format(kriging_model=kriging_model)
    line_8 = line_8.format(kriging_correction=kriging_correction)
    line_9 = line_9.format(secs_timestep=secs_timestep)
    line_11 = line_11.format(datetime=starting_date)
    line_12 = line_12.format(basin=basin)

    file = open(pathout+"kriging_params.in","w+")
    [file.writelines(el + "\n") for el in [line_1, line_2, line_3, line_4, line_5, line_6, line_7, line_8, line_9, line_10, line_11, line_12, line_13]]

In [ ]:
#### SETUP ####
wdir = "D:\\projects\\kriging\\Scripts\\unibz\\test_python\\"
config_path = wdir + "etc\\config\\"
input_path = wdir + "input\\"

# KRIGING MODEL SETUP
exe_path = wdir 
# exe_name = "2020_06_kriging_PT.exe"
exe_name = "kriging_v3"

In [ ]:
inputs = glob.glob( input_path + "*.json" )

In [ ]:
for input in inputs:
    setup_file = open(inputs[0])
    setup_sim = json.load( setup_file )

    variable = setup_sim["variable"]

    if variable == "temperature":
        config_file = open( config_path + "temperature.json")
    elif variable == "precipitation":
        config_file = open( config_path + "precipitation.json")
    params = json.load( config_file )

    simulation_type = setup_sim["simulation_type"]

    basin = setup_sim["basin"]
    secs_timestep = setup_sim["secs_timestep"]
    number_of_stations = setup_sim["number_of_stations"]
    kriging_type = setup_sim["kriging_type"]
    kriging_model = params["kriging_model"]
    kriging_correction = setup_sim["kriging_correction"]

    number_of_cells = len(setup_sim["cells"])

    start_datetime_str = setup_sim["start_datetime"]
    start_datetime = dt.datetime.strptime(start_datetime_str, '%Y-%m-%d %H:%M:%S')
    end_datetime_str = setup_sim["end_datetime"]
    end_datetime = dt.datetime.strptime(end_datetime_str, '%Y-%m-%d %H:%M:%S')
    dates = pd.date_range(start_datetime, end_datetime, freq='h')
    steps = len(dates)

    weather_data_path = wdir + setup_sim["weather_data_path"]

    ### RUN KRIGING on a POINT ###

    # #TOP
    # station_name = "top"
    # east = 691258.2633
    # west = 5175335.014
    # elevation = 2161.344

    # #BOTTOM
    # station_name = "bottom"
    # east = 645258.2633
    # west = 5165335.014
    # elevation = 581.2944

    # #RANDOM
    # station_name = "random"
    # east = 641258.2633
    # west = 5133335.014
    # elevation = 1755.157

    ##################################

    ### RUN CROSS VALIDATION ###

    # #ANTERSELVA DI SOPRA
    # station_name = "Anterselva"
    # station_id = 16 #Anterselva
    # east = 737506.6
    # west = 5195485
    # elevation = 1320


In [ ]:
    path_kriging_inputs = wdir + basin + "\\input\\"
    path_kriging_outputs = wdir + basin + "\\output\\"

In [ ]:
    for i in range(number_of_cells):

        station_name = setup_sim["cells"][i]["station_name"]
        station_id = setup_sim["cells"][i]["station_id"]
        east = setup_sim["cells"][i]["east"]
        west = setup_sim["cells"][i]["west"]
        elevation = setup_sim["cells"][i]["elevation"]

        output_name = simulation_type + "_" + station_name + "_" + kriging_type + "_" + start_datetime.strftime(format='%Y%m%d') + "_" + end_datetime.strftime(format='%Y%m%d')

        # OUTPUT DIRECTORY
        output_dir = wdir + "output\\" + variable + "\\" + simulation_type + "\\" + kriging_type + "\\" + kriging_correction + "\\"
        mkNestedDir(output_dir)

        ###########################################################

        if bool( setup_sim['generate_inputs'] ) == True:
            dataset  = pd.DataFrame(index=dates)
            metadata = pd.DataFrame(index=range(0,3))
            big_frame = pd.DataFrame()

            big_frame = pd.DataFrame()

            # percentage of no data allowed - range (0,1]
            perc_filter = 1
                        
            # Get list of files available
            files = os.listdir( weather_data_path )
                                    
            # Loop over weather stations
            for filename in files:
                    
                # Get station ID
                IDstation = filename.split('.')[0]

                # print(int(IDstation))
                if simulation_type == "CV":
                    if int(IDstation) == int(station_id):
                        print("Excluded:" + str(IDstation))
                        continue

                # Reading data
                data = pd.read_csv( weather_data_path + filename, index_col=0, parse_dates=True, names=['labels','values'])
                                
                # Extract time series in the required time window
                ts = data.loc[ start_datetime_str:end_datetime_str ]     
                                
                # Count no data
                n_no_data = len(ts[ts['values'] == -999])
                                
                if n_no_data > perc_filter*len(ts):
                    continue
                else:
                    # Extract metadata
                    metadata[IDstation] = data.iloc[1:4].values
                    # Populate the overall matrix of data
                    dataset[IDstation] = ts.values

                    big_frame=pd.concat([metadata,dataset])            

            station_number = len(big_frame.columns)
            buckets = [0] * (station_number - 3)
            buckets.insert(0,secs_timestep)
            buckets.insert(0,station_number)
            buckets.insert(0,steps)

            big_frame_meta = pd.DataFrame(data=[buckets],columns=big_frame.columns.values)
            big_frame_id = pd.DataFrame(data=[big_frame.columns.values],columns=big_frame.columns.values)
            big_frame = pd.concat([ pd.concat([big_frame_meta,big_frame_id]),big_frame ])

            if variable == "temperature":
                big_frame.to_csv( path_kriging_inputs + "TMEAN_" + basin + ".in", index=False, header=False )
            elif variable == "precipitation":
                big_frame.to_csv( path_kriging_inputs + "P_" + basin + ".in", index=False, header=False )
            
        write_params(variable, number_of_stations, kriging_type, kriging_model, kriging_correction, params["model_nugget"], params["model_psill"], params["model_range"],
        secs_timestep, start_datetime.strftime(format='%Y/%m/%d %H:%M'), basin, path_kriging_inputs)

        pathout_subcell = wdir + "AltoAdige\\input\\1km_AA\\"
        write_subcells(1, station_id, 30, east, west, elevation, pathout_subcell)
        write_geometry(1, 30, pathout_subcell)

        ### RUN THE MODEL
        path = exe_path
        os.chdir(path)
        os.system(exe_name)

        if variable == "temperature":
            exe_output_name = path_kriging_outputs + str(start_datetime.year) + "_" + str(start_datetime.month) + "\\TMEAN_" + basin + ".krig"
        elif variable == "precipitation":
            exe_output_name = path_kriging_outputs + str(start_datetime.year) + "_" + str(start_datetime.month) + "\\P_" + basin + ".krig"

        if output_dir == None:
            output_dir = os.path.dirname( exe_output_name )

        output_file = output_dir + output_name + ".csv"
        print( "Moving output to: " + output_file )
        try:
            os.rename( exe_output_name, output_file )
        except FileExistsError:
            os.remove( output_file )
            os.rename( exe_output_name, output_file )

In [ ]:
# ### POST-PROCESSING ###

# start_datetime_str = '2013-01-01 00:00:00'
# end_datetime_str = '2014-12-31 23:00:00'

# resample = "h"
# # resample = "d"
# # resample = "MS"

# model_output = pd.read_csv( output_file )

# model_output.index = dates
# model_output = model_output[start_datetime_str:end_datetime_str]
# model_output = model_output.resample(resample).mean()
# model_output = model_output.iloc[:,0]

# data_length = len(model_output)

In [ ]:
# obs_data_name = weather_data_path + str(station_id) + ".txt"
# print("Opening: " + obs_data_name)

# obs_data = pd.read_csv( obs_data_name, index_col=0,parse_dates=True, skiprows=4, names=['values'] )

# obs_data = obs_data[start_datetime_str:end_datetime_str]
# obs_data[obs_data == -999] = None 
# obs_data = obs_data.resample(resample).mean()

# # obs_data = pd.to_numeric(obs_data, errors='coerce')

In [ ]:
# import numpy.ma as ma

# rmse = np.sqrt((1/data_length) *  np.nansum((obs_data.T.values - model_output.values)**2 ))  
# cc = ma.corrcoef(ma.masked_invalid(model_output.values), ma.masked_invalid(obs_data.T.values))
# mae = (np.nansum(np.abs(obs_data.T.values - model_output.values) ))/data_length

# print(rmse)